# Best way to jsonify numpy arrray


In [6]:
import json
import numpy as np

ARRAY_SIZE = 1536

array = np.random.rand(ARRAY_SIZE).astype(np.float64)

print('numpy => list:')
%timeit array.tolist()
print('numpy => list => json:')
%timeit json.dumps(array.tolist())

json.dumps(array.tolist())


numpy => list:
14.4 µs ± 87.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
numpy => list => json:
498 µs ± 761 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


'[0.143825201536814, 0.05372366053603639, 0.047997948188230555, 0.5088893587175664, 0.032148406600217005, 0.22952407938720687, 0.5403204473769251, 0.7248464257064559, 0.3051087359727549, 0.8761658005800299, 0.2306369450068938, 0.7486871405200233, 0.5500706710754584, 0.4984798473354415, 0.17175967045966645, 0.7392596482254579, 0.4707071278812034, 0.5027167019527079, 0.1844303315320659, 0.06378808536708758, 0.6345766024537646, 0.7932201472233285, 0.9972512366361884, 0.939277393201935, 0.5160093856585364, 0.8646171176653339, 0.07507174813500461, 0.08602046541038855, 0.3757425280042951, 0.29599760476836623, 0.7145796023663624, 0.6545188911889335, 0.7472190690763687, 0.66243967903587, 0.29695681786522354, 0.10578361374999212, 0.11669821848974027, 0.41732465682169695, 0.7186780724787336, 0.41332187392177044, 0.4288512522092306, 0.8178965823887622, 0.5272309042845809, 0.3139756186697935, 0.510760214149774, 0.10183840939407607, 0.7048632401692003, 0.7023749386560817, 0.0398751004213298, 0.1635

In [7]:
import orjson
%timeit orjson.dumps(array, option=orjson.OPT_SERIALIZE_NUMPY,)
orjson.dumps(array, option=orjson.OPT_SERIALIZE_NUMPY,)

def to_json(array: np.ndarray):
    return orjson.dumps(array, option=orjson.OPT_SERIALIZE_NUMPY,).decode('utf-8')
print('numpy => json (via orjson)')
%timeit to_json(array)
to_json(array)

31.9 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
numpy => json (via orjson)
32.8 µs ± 80.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


'[0.143825201536814,0.05372366053603639,0.047997948188230555,0.5088893587175664,0.032148406600217005,0.22952407938720687,0.5403204473769251,0.7248464257064559,0.3051087359727549,0.8761658005800299,0.2306369450068938,0.7486871405200233,0.5500706710754584,0.4984798473354415,0.17175967045966645,0.7392596482254579,0.4707071278812034,0.5027167019527079,0.1844303315320659,0.06378808536708758,0.6345766024537646,0.7932201472233285,0.9972512366361884,0.939277393201935,0.5160093856585364,0.8646171176653339,0.07507174813500461,0.08602046541038855,0.3757425280042951,0.29599760476836623,0.7145796023663624,0.6545188911889335,0.7472190690763687,0.66243967903587,0.29695681786522354,0.10578361374999212,0.11669821848974027,0.41732465682169695,0.7186780724787336,0.41332187392177044,0.4288512522092306,0.8178965823887622,0.5272309042845809,0.3139756186697935,0.510760214149774,0.10183840939407607,0.7048632401692003,0.7023749386560817,0.0398751004213298,0.16353847694176882,0.7961042841298456,0.25612216672578

In [10]:
# extend the json.JSONEncoder class
from typing import Any
import json
import orjson

class FastJSONEncoder(json.JSONEncoder):

    def encode(self, o: Any) -> str:
        return orjson.dumps(o, option=orjson.OPT_SERIALIZE_NUMPY,).decode('UTF-8')

%timeit json.dumps({'x': array}, cls=FastJSONEncoder)
json.dumps({'x': array}, cls=FastJSONEncoder)

33.3 µs ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


'{"x":[0.143825201536814,0.05372366053603639,0.047997948188230555,0.5088893587175664,0.032148406600217005,0.22952407938720687,0.5403204473769251,0.7248464257064559,0.3051087359727549,0.8761658005800299,0.2306369450068938,0.7486871405200233,0.5500706710754584,0.4984798473354415,0.17175967045966645,0.7392596482254579,0.4707071278812034,0.5027167019527079,0.1844303315320659,0.06378808536708758,0.6345766024537646,0.7932201472233285,0.9972512366361884,0.939277393201935,0.5160093856585364,0.8646171176653339,0.07507174813500461,0.08602046541038855,0.3757425280042951,0.29599760476836623,0.7145796023663624,0.6545188911889335,0.7472190690763687,0.66243967903587,0.29695681786522354,0.10578361374999212,0.11669821848974027,0.41732465682169695,0.7186780724787336,0.41332187392177044,0.4288512522092306,0.8178965823887622,0.5272309042845809,0.3139756186697935,0.510760214149774,0.10183840939407607,0.7048632401692003,0.7023749386560817,0.0398751004213298,0.16353847694176882,0.7961042841298456,0.256122166

In [11]:
json.dumps(array, cls=FastJSONEncoder)

'[0.143825201536814,0.05372366053603639,0.047997948188230555,0.5088893587175664,0.032148406600217005,0.22952407938720687,0.5403204473769251,0.7248464257064559,0.3051087359727549,0.8761658005800299,0.2306369450068938,0.7486871405200233,0.5500706710754584,0.4984798473354415,0.17175967045966645,0.7392596482254579,0.4707071278812034,0.5027167019527079,0.1844303315320659,0.06378808536708758,0.6345766024537646,0.7932201472233285,0.9972512366361884,0.939277393201935,0.5160093856585364,0.8646171176653339,0.07507174813500461,0.08602046541038855,0.3757425280042951,0.29599760476836623,0.7145796023663624,0.6545188911889335,0.7472190690763687,0.66243967903587,0.29695681786522354,0.10578361374999212,0.11669821848974027,0.41732465682169695,0.7186780724787336,0.41332187392177044,0.4288512522092306,0.8178965823887622,0.5272309042845809,0.3139756186697935,0.510760214149774,0.10183840939407607,0.7048632401692003,0.7023749386560817,0.0398751004213298,0.16353847694176882,0.7961042841298456,0.25612216672578